## E-commerce — Выявление профилей потребления в магазине «Пока все ещё тут».

### Цель - выработка таргетированных маркетинговых стратегий отдельно для каждого сегмента покупателей  
- формирование для каждого сегмента специальных предолжений  
    * товаров, которые интересны пользователям  
    * дополняющих товаров, которые могли бы пользователя заинтересовать
- предложение специальных условий обслуживания  и прочее

In [6]:
import pandas as pd
data = pd.read_csv('/Users/kbzunder/Downloads/ecommerce_dataset.csv')
data.tail(20)

,date,customer_id,order_id,product,quantity,price
6717,2019103009,344aa778-e436-419e-a9c6-9b8f37b7c1df,73137,Сумка-тележка 2-х колесная Gimi Argo синяя,1,1087.0
6718,2019103009,6406e169-0238-44c4-8ab5-e20190778679,73138,Пеларгония розебудная Queen Ingrid укорененный...,3,149.0
6719,2019103010,55f5aaea-0a3f-46c0-8677-e0d5be2d19ee,73140,"Сушилка для белья потолочная Лиана 1,3 м 1703002",1,509.0
6720,2019103010,55f5aaea-0a3f-46c0-8677-e0d5be2d19ee,73141,Сушилка для белья потолочно-настенная Gimi Lif...,1,1349.0
6721,2019103012,78b878f7-7f2d-4069-bcf0-0e3531583696,73142,Искусственный цветок Ромашка пластик 45 см,4,22.0
6722,2019103013,8a83e422-37fc-4333-b26a-e3a0e89d197b,73143,Полки QWERTY Италия белый/серый 40х14х10 см 35...,1,1949.0
6723,2019103015,12157e93-5760-473d-bb0d-ac8b3be1c5ca,73144,Эвкалипт Гунни d-14 см h-70 см,1,524.0
6724,2019103018,7bd75579-e672-439a-9a55-e2a7072b5466,73146,Фиалка d-9 см синяя с белым рифленая,1,119.0
6725,2019103019,ff0c73c3-6b62-4e67-8e12-bb83ed1851d5,73147,Сушилка для белья на ванну FREUDENBERG (GIMI) ...,1,899.0
6726,2019103021,cb65d08a-dae7-4890-aef0-bb9f79055e02,73148,Каланхое каландива малиновое d-7 см,1,127.0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6737 entries, 0 to 6736
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         6737 non-null   int64  
 1   customer_id  6737 non-null   object 
 2   order_id     6737 non-null   int64  
 3   product      6737 non-null   object 
 4   quantity     6737 non-null   int64  
 5   price        6737 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 315.9+ KB


В нашем распоряжении ограниченный набор данных, не включающий в себя информцию о возрасте, поле, географическом положении покупателя, поэтому  возможности сегментации ограничены предположительно следюущими типами:   
- покупатель, совершивший одну покупку / вернувшийся покупатель  
- количество покупок
- среднее время между покупками  
- LTV / средний чек  
- время совершения покупки (сезон/будни/выходные/время дня)  
- тип покупки  

Поскольку столбец product не содержит точного указания на тип товара, предполагаю использовать NLP модель для формирования признаков и последующего проведения кластеризации для выявления групп товаров.

### План работы над проектом  
1. Знакомство с данными и предобработка  
    - изменение типов данных, выявление дубликатов
2. Исследование данных
    - аномалии, закономерности
3. Разделение пользователей на сегменты, анализ ключевых метрик внутри сегмента: LTV, средний чек, время между покупками
4. Формулировка и проверка  гипотез:
    - H0: LTV/средний чек/время между покупками  одинаковы между сегментами
    - H1: LTV/средний чек/время между покупками различны между сегментами
5. Анализ результатов исследования  
6. Формулирование выводов и предложений  
7. Формирования дашборда и презентации для визуализации выводов и предложений